In [28]:
import os
import getpass
from typing import Literal
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langchain_anthropic import ChatAnthropic
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode
from tenacity import retry, stop_after_attempt, wait_exponential

# Environment setup
def _set_env(var: str):
    """Set up environment variables securely"""
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

# Tool definitions
@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 60 degrees and foggy."
    elif location.lower() in ["bal", "baltimore"]:
        return "It's 50 degrees and rainy."
    else:
        return "It's 90 degrees and sunny."

@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "nyc, sf"

@tool
def add_list_item(item: str, state: MessagesState):
    """Add an item to the list array stored in state"""
    if "list" not in state:
        state["list"] = []
    state["list"].append(item)
    return f"Added {item} to the list. Current list: {', '.join(state['list'])}"

@tool
def get_list_items(state: MessagesState):
    """Get all items currently in the list"""
    if "list" not in state:
        return "The list is empty"
    return f"Current list items: {', '.join(state['list'])}"

@tool
def all_tools():
    """Lists all available tools and their descriptions"""
    tool_descriptions = [
        {
            "name": "get_weather",
            "description": "Call to get the current weather for a specific location",
        },
        {
            "name": "get_coolest_cities",
            "description": "Get a list of coolest cities",
        },
        {
            "name": "all_tools",
            "description": "Lists all available tools and their descriptions",
        },
        {
            "name": "add_list_item",
            "description": "Add an item to the list stored in state",
        },
        {
            "name": "get_list_items", 
            "description": "Get all items currently in the list",
        },
    ]
    return "\n".join([f"- {t['name']}: {t['description']}" for t in tool_descriptions])

# Setup tools and model
tools = [get_weather, get_coolest_cities, all_tools, add_list_item, get_list_items]
tool_node = ToolNode(tools)

# Add retry decorator to handle rate limits
@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=4, max=10))
def call_model_with_retry(model, messages):
    """Call model with retry logic for rate limits"""
    return model.invoke(messages)

model_with_tools = ChatAnthropic(
    model="claude-3-5-sonnet-20241022", 
    temperature=0
).bind_tools(tools)

# Workflow functions
def should_continue(state: MessagesState):
    """Determine if we should continue to tools or end"""
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END

def call_model(state: MessagesState):
    """Process messages through the model"""
    messages = state["messages"]
    response = call_model_with_retry(model_with_tools, messages)
    return {"messages": [response]}

# Setup workflow graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)
workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

# Compile the application
app = workflow.compile()

# Example usage function
def process_message(message: str):
    """Process a single message through the workflow"""
    for chunk in app.stream(
        {"messages": [("human", message)]}, 
        stream_mode="values"
    ):
        chunk["messages"][-1].pretty_print()

# Example usage:
# process_message("what's the weather in sf?")
# process_message("add carrots to the list")

In [32]:
process_message("whats in the list")

================================ Human Message =================================

whats in the list
================================== Ai Message ==================================

[{'text': "I'll help you check what items are currently in the list using the get_list_items function.", 'type': 'text'}, {'id': 'toolu_012JEzqMjKHNqGK7qqeBWMSs', 'input': {'state': {'messages': []}}, 'name': 'get_list_items', 'type': 'tool_use'}]
Tool Calls:
  get_list_items (toolu_012JEzqMjKHNqGK7qqeBWMSs)
 Call ID: toolu_012JEzqMjKHNqGK7qqeBWMSs
  Args:
    state: {'messages': []}
================================= Tool Message =================================
Name: get_list_items

The list is empty
================================== Ai Message ==================================

The list is currently empty. You can add items to it using commands like "add X to the list" if you'd like.
